In [77]:
import pandas as pd
import os
from datetime import datetime, timedelta
import glob
import numpy as np

In [78]:
# Process all tags
tags = [
    "flu", "cough", "fever", "headache", "lagnat", "rashes",
    "sipon", "ubo", "ecq", "face-shield", "Frontliners",
    "masks", "Quarantine", "social-distancing", "work-from-home"
]

In [79]:
def load_csv_file(filepath):
    """Load a CSV file and remove the 'isPartial' column."""
    df = pd.read_csv(filepath)
    return df.drop('isPartial', axis=1, errors='ignore')

In [80]:
def get_correction_factor(first_file_df, second_file_df, tag_column):
    """Calculate correction factor between two consecutive files."""
    # Get all dates that appear in both files
    common_dates = list(set(first_file_df['date']) & set(second_file_df['date']))
    common_dates.sort()  # Ensure dates are in order
    
    # For 30-day window, we need 29 quotients
    required_quotients = 29
    
    if len(common_dates) < required_quotients:
        print(f"Warning: Only found {len(common_dates)} common dates, needed {required_quotients}")
        return 1.0
    
    # Take only the first required_quotients number of dates
    common_dates = common_dates[:required_quotients]
    
    quotients = []
    for date in common_dates:
        first_value = first_file_df[first_file_df['date'] == date][tag_column].iloc[0]
        second_value = second_file_df[second_file_df['date'] == date][tag_column].iloc[0]
        
        try:
            if second_value == 0 or np.isnan(first_value) or np.isnan(second_value):
                quotient = 1.0
            else:
                quotient = first_value / second_value
                if not np.isreal(quotient) or np.isinf(quotient) or np.isnan(quotient):
                    quotient = 1.0
        except:
            quotient = 1.0
            
        quotients.append(quotient)
    
    correction_factor = sum(quotients) / required_quotients
    return correction_factor

In [81]:
def process_tag_files(tag):
    """Process all CSV files for a given tag."""
    tag_column = tag.replace('-', ' ')
    folder_path = f"../../gt_raw_daily30daywindow_volumes/{tag}"
    csv_files = sorted(glob.glob(os.path.join(folder_path, "*.csv")))
    
    if not csv_files:
        print(f"No CSV files found for tag: {tag}")
        return None
    
    # Initialize result DataFrame with first file's initial days
    first_file_df = load_csv_file(csv_files[0])
    initial_days = 29  # 29 days for 30-day window
    result_df = first_file_df.head(initial_days).copy()
    
    # Define cutoff date
    CUTOFF_DATE = '2021-03-15'
    
    # Keep track of the last processed date
    last_processed_date = None
    
    # Process files in pairs
    for i in range(len(csv_files) - 1):
        current_file = csv_files[i]
        next_file = csv_files[i + 1]
        
        print(f"Processing files: {os.path.basename(current_file)} and {os.path.basename(next_file)}")
        
        # Load the CSV files
        current_df = load_csv_file(current_file)
        next_df = load_csv_file(next_file)
        
        # Get all dates from both files
        all_dates = sorted(list(set(current_df['date'].tolist() + next_df['date'].tolist())))
        
        # If we have a gap from last processed date, fill it using the last correction factor
        if last_processed_date is not None:
            last_processed_datetime = datetime.strptime(last_processed_date, '%Y-%m-%d')
            current_start_datetime = datetime.strptime(min(all_dates), '%Y-%m-%d')
            
            if (current_start_datetime - last_processed_datetime).days > 1:
                # Generate missing dates
                missing_date = last_processed_datetime + timedelta(days=1)
                while missing_date < current_start_datetime:
                    missing_date_str = missing_date.strftime('%Y-%m-%d')
                    
                    # Use the last available value and correction factor
                    if missing_date_str <= CUTOFF_DATE:
                        last_value = result_df.iloc[-1][tag_column]
                        
                        # Add to result DataFrame
                        new_row = pd.DataFrame({
                            'date': [missing_date_str],
                            tag_column: [last_value]  # Use last value for gap filling
                        })
                        result_df = pd.concat([result_df, new_row], ignore_index=True)
                    
                    missing_date += timedelta(days=1)
        
        # Calculate correction factor and process normal dates
        correction_factor = get_correction_factor(current_df, next_df, tag_column)
        
        target_dates = sorted(next_df['date'].unique())
        if len(target_dates) >= 30:  # 30-day window
            target_date = target_dates[29]  # Get the 30th day
            
            if target_date <= CUTOFF_DATE:
                original_value = next_df[next_df['date'] == target_date][tag_column].iloc[0]
                corrected_value = original_value * correction_factor
                
                new_row = pd.DataFrame({
                    'date': [target_date],
                    tag_column: [corrected_value]
                })
                result_df = pd.concat([result_df, new_row], ignore_index=True)
                last_processed_date = target_date
    
    # Fill any remaining gaps until CUTOFF_DATE
    if last_processed_date:
        last_processed_datetime = datetime.strptime(last_processed_date, '%Y-%m-%d')
        cutoff_datetime = datetime.strptime(CUTOFF_DATE, '%Y-%m-%d')
        
        current_date = last_processed_datetime + timedelta(days=1)
        while current_date <= cutoff_datetime:
            current_date_str = current_date.strftime('%Y-%m-%d')
            
            # Use the last available value
            last_value = result_df.iloc[-1][tag_column]
            
            new_row = pd.DataFrame({
                'date': [current_date_str],
                tag_column: [last_value]  # Use last value for final gap filling
            })
            result_df = pd.concat([result_df, new_row], ignore_index=True)
            
            current_date += timedelta(days=1)
    
    # Ensure the DataFrame is sorted by date and has no duplicates
    result_df = result_df.sort_values('date').drop_duplicates(subset='date', keep='first')
    return result_df

In [82]:
# Process each tag
for tag in tags:
    print(f"\nProcessing tag: {tag}")
    result_df = process_tag_files(tag)
    
    if result_df is not None:
        output_filename = f"{tag}_msv_stitched_30day.csv"
        result_df.to_csv(output_filename, index=False)
        print(f"Saved processed data to {output_filename}")


Processing tag: flu
Processing files: 2020 03 16.csv and 2020 03 17.csv
Processing files: 2020 03 17.csv and 2020 03 18.csv
Processing files: 2020 03 18.csv and 2020 03 19.csv
Processing files: 2020 03 19.csv and 2020 03 20.csv
Processing files: 2020 03 20.csv and 2020 03 21.csv
Processing files: 2020 03 21.csv and 2020 03 22.csv
Processing files: 2020 03 22.csv and 2020 03 23.csv
Processing files: 2020 03 23.csv and 2020 03 24.csv
Processing files: 2020 03 24.csv and 2020 03 25.csv
Processing files: 2020 03 25.csv and 2020 03 26.csv
Processing files: 2020 03 26.csv and 2020 03 27.csv
Processing files: 2020 03 27.csv and 2020 03 28.csv
Processing files: 2020 03 28.csv and 2020 03 29.csv
Processing files: 2020 03 29.csv and 2020 03 30.csv
Processing files: 2020 03 30.csv and 2020 03 31.csv
Processing files: 2020 03 31.csv and 2020 04 01.csv
Processing files: 2020 04 01.csv and 2020 04 02.csv
Processing files: 2020 04 02.csv and 2020 04 03.csv
Processing files: 2020 04 03.csv and 2020 0